# FraSoHome — Notebook 3: Limpieza y estandarización (Data Cleaning)

**Objetivo formativo:** partir de los CSV “raw” (con errores intencionales) y aplicar una **limpieza reproducible** y **documentada** para dejar los datasets en condiciones de integrarse y analizarse.

En este notebook:

- Cargamos todos los **archivos origen** como texto (`dtype=str`) para no “perder” errores.
- Estandarizamos **IDs**, **texto**, **fechas** (formatos mixtos) y **números** (€, coma/punto, miles…).
- Tratamos **nulos**, **duplicados** y **anomalías** (cantidades/stock negativos, fechas futuras, etc.).
- Separamos/etiquetamos **registros problemáticos** para que los alumnos vean *qué se corrige* y *qué se descarta*.
- Exportamos datasets `*_clean.csv` + logs de limpieza para el siguiente notebook (integración / fact table).

> Importante: En un proyecto real, algunas decisiones (imputación, descarte, reglas de negocio) se validan con negocio. Aquí se proponen reglas **didácticas**.

In [16]:
# === 1) Imports y configuración ===
from __future__ import annotations

from dataclasses import dataclass
from pathlib import Path
from typing import Dict, List, Tuple, Optional, Iterable

import numpy as np
import pandas as pd
import re
import json

pd.set_option("display.max_columns", 200)
pd.set_option("display.width", 140)

DATA_DIR = Path(r"..\data\raw")  # Ajusta si tus CSV están en otra carpeta (p.ej. Path('data'))
OUT_DIR =Path(".\outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)

FILES = {
    "crm": "crm.csv",
    "productos": "productos.csv",
    "tiendas": "tiendas.csv",
    "pedidos": "pedidos.csv",
    "lineas_pedido": "lineas_pedido.csv",
    "devoluciones_online": "devoluciones_online.csv",
    "ventas_pos": "ventas_pos.csv",
    "devoluciones_tienda": "devoluciones_tienda.csv",
    "pagos_tienda": "pagos_tienda.csv",
    "stock_diario": "stock_diario.csv",
    # fact_transacciones es derivado; se limpia mejor en el notebook de integración si aplica
}

# Cutoff didáctico para detectar fechas futuras "sospechosas"
MAX_REASONABLE_DATE = pd.Timestamp("2026-01-01")

<>:17: SyntaxWarning: invalid escape sequence '\o'
<>:17: SyntaxWarning: invalid escape sequence '\o'
C:\Users\anton\AppData\Local\Temp\ipykernel_25280\919493428.py:17: SyntaxWarning: invalid escape sequence '\o'
  OUT_DIR =Path(".\outputs")


In [4]:
# === 2) Utilidades genéricas de limpieza ===

def load_csv_raw(path: Path) -> pd.DataFrame:
    """Carga un CSV en modo 'raw' para no perder errores.

    - dtype=str: todo como texto
    - keep_default_na=False: evita que 'N/A', 'NULL', etc. se conviertan automáticamente
    - na_values: definimos nuestros nulos
    """
    return pd.read_csv(
        path,
        dtype=str,
        encoding="utf-8",
        keep_default_na=False,
        na_values=["", " ", "NA", "N/A", "NULL", "null", "None", "nan", "NaN"],
    )

def standardize_column_names(df: pd.DataFrame) -> pd.DataFrame:
    """Normaliza nombres de columnas: lower, strip, espacios->underscore."""
    df = df.copy()
    df.columns = (
        df.columns
        .astype(str)
        .str.strip()
        .str.lower()
        .str.replace(r"\s+", "_", regex=True)
    )
    return df

def strip_strings(df: pd.DataFrame, cols: Optional[Iterable[str]] = None) -> pd.DataFrame:
    """Hace strip() de columnas string seleccionadas (o todas si cols=None)."""
    df = df.copy()
    if cols is None:
        cols = df.columns
    for c in cols:
        if c in df.columns:
            df[c] = df[c].astype(str).str.strip()
            # Convertimos cadenas vacías en NaN (pero manteniendo el dtype object)
            df[c] = df[c].replace({"": np.nan, "nan": np.nan, "None": np.nan})
    return df

def standardize_id(series: pd.Series, upper: bool = True) -> pd.Series:
    """Estandariza IDs (trim + mayúsculas opcional)."""
    s = series.astype(str).str.strip()
    s = s.replace({"": np.nan, "nan": np.nan, "None": np.nan})
    if upper:
        s = s.str.upper()
    return s

def normalize_text(series: pd.Series) -> pd.Series:
    """Normaliza texto para categóricas: trim y espacios múltiples."""
    s = series.astype(str).str.strip()
    s = s.replace({"": np.nan, "nan": np.nan, "None": np.nan})
    s = s.str.replace(r"\s+", " ", regex=True)
    return s

def drop_exact_duplicates(df: pd.DataFrame) -> Tuple[pd.DataFrame, int]:
    """Elimina duplicados exactos (toda la fila)."""
    before = len(df)
    out = df.drop_duplicates()
    return out, before - len(out)

def dedupe_on_key(df: pd.DataFrame, key_cols: List[str], sort_col: Optional[str] = None) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """Deduplica por clave, opcionalmente quedándose con el registro 'más reciente' según sort_col.

    Retorna:
    - df_dedup: dataset deduplicado
    - df_dups: filas descartadas (duplicadas por key)
    """
    df = df.copy()
    missing = [c for c in key_cols if c not in df.columns]
    if missing:
        raise ValueError(f"Faltan columnas key: {missing}")

    if sort_col and sort_col in df.columns:
        # Si sort_col es datetime ya tipado, mejor; si no, se ordenará lexicalmente.
        df = df.sort_values(sort_col, ascending=False, na_position="last")

    dup_mask = df.duplicated(subset=key_cols, keep="first")
    df_dups = df.loc[dup_mask].copy()
    df_dedup = df.loc[~dup_mask].copy()
    return df_dedup, df_dups

In [5]:
# === 3) Parseo robusto de fechas (con meses en español) ===

SPANISH_MONTHS = {
    "enero": "01", "febrero": "02", "marzo": "03", "abril": "04",
    "mayo": "05", "junio": "06", "julio": "07", "agosto": "08",
    "septiembre": "09", "setiembre": "09", "octubre": "10",
    "noviembre": "11", "diciembre": "12",
}

def _normalize_spanish_date_text(s: str) -> str:
    """Convierte textos tipo '10 de Enero de 2023' -> '10/01/2023' (con hora si viene)."""
    if s is None:
        return s
    x = str(s).strip()
    if not x:
        return x

    # Ej: '10 de Enero de 2023' / '27 de Julio de 2025 13:20'
    m = re.match(
        r"^(\d{1,2})\s+de\s+([A-Za-zÁÉÍÓÚáéíóúñÑ]+)\s+de\s+(\d{4})(?:\s+(\d{1,2}:\d{2}(?::\d{2})?))?$",
        x,
        flags=re.IGNORECASE,
    )
    if m:
        day = m.group(1).zfill(2)
        month_name = m.group(2).lower()
        month = SPANISH_MONTHS.get(month_name, None)
        year = m.group(3)
        time_part = m.group(4)
        if month:
            if time_part:
                return f"{day}/{month}/{year} {time_part}"
            return f"{day}/{month}/{year}"

    return x

def parse_datetime_es(series: pd.Series, dayfirst: bool = True) -> pd.Series:
    """Parsea fechas con formatos mixtos (ISO, DD/MM, DD/MM/YY, texto español)."""
    s = series.copy()
    s = s.replace({"": np.nan, "nan": np.nan, "None": np.nan})
    s = s.astype(str)
    s_norm = s.apply(_normalize_spanish_date_text)

    # Primero intentamos parse general
    dt1 = pd.to_datetime(s_norm, errors="coerce", dayfirst=dayfirst, utc=False)

    return dt1

def date_parse_report(series: pd.Series, name: str) -> pd.DataFrame:
    """Reporte rápido de parseo de fechas."""
    dt_ser = parse_datetime_es(series)
    total = len(series)
    ok = dt_ser.notna().sum()
    bad = total - ok
    return pd.DataFrame([{
        "campo": name,
        "total": total,
        "parse_ok": int(ok),
        "parse_bad": int(bad),
        "parse_ok_pct": round(ok/total*100, 2) if total else 0.0
    }])

In [6]:
# === 4) Parseo robusto de números (€, coma/punto, miles) ===

def _clean_numeric_str(x: str) -> str:
    if x is None:
        return ""
    s = str(x).strip()
    if not s:
        return ""
    # Quitar moneda y letras comunes
    s = re.sub(r"(EUR|€|\s)", "", s, flags=re.IGNORECASE)
    # Mantener solo dígitos, separadores y signo
    s = re.sub(r"[^0-9,\.\-]", "", s)
    return s

def parse_numeric(series: pd.Series) -> pd.Series:
    """Convierte series de texto a float, intentando inferir separador decimal."""
    s = series.replace({"": np.nan, "nan": np.nan, "None": np.nan}).astype(str)
    cleaned = s.apply(_clean_numeric_str)

    def to_float(v: str) -> float:
        if v is None:
            return np.nan
        v = str(v).strip()
        if v == "" or v.lower() in {"nan", "none"}:
            return np.nan

        # Si tiene '.' y ',', asumimos que el último separador es decimal
        if "," in v and "." in v:
            last_comma = v.rfind(",")
            last_dot = v.rfind(".")
            if last_comma > last_dot:
                # coma decimal, puntos miles
                v2 = v.replace(".", "").replace(",", ".")
            else:
                # punto decimal, comas miles
                v2 = v.replace(",", "")
        elif "," in v:
            # coma decimal
            v2 = v.replace(".", "").replace(",", ".")
        else:
            v2 = v

        try:
            return float(v2)
        except Exception:
            return np.nan

    return cleaned.apply(to_float)

def parse_int(series: pd.Series) -> pd.Series:
    """Convierte a entero (nullable Int64), usando parse_numeric como base."""
    f = parse_numeric(series)
    # Redondeo defensivo
    out = np.floor(f).astype("float")
    return pd.Series(out).astype("Int64")

def numeric_parse_report(series: pd.Series, name: str) -> pd.DataFrame:
    f = parse_numeric(series)
    total = len(series)
    ok = f.notna().sum()
    bad = total - ok
    return pd.DataFrame([{
        "campo": name,
        "total": total,
        "parse_ok": int(ok),
        "parse_bad": int(bad),
        "parse_ok_pct": round(ok/total*100, 2) if total else 0.0
    }])

In [7]:
# === 5) Checks básicos de calidad / consistencia ===

def basic_shape_report(df: pd.DataFrame, name: str) -> pd.DataFrame:
    return pd.DataFrame([{
        "dataset": name,
        "rows": df.shape[0],
        "cols": df.shape[1],
        "null_cells": int(df.isna().sum().sum()),
        "null_cells_pct": round(df.isna().sum().sum() / (df.shape[0]*df.shape[1]) * 100, 2) if df.shape[0] and df.shape[1] else 0.0,
        "exact_dups": int(df.duplicated().sum()),
    }])

def pk_duplicate_report(df: pd.DataFrame, name: str, key_cols: List[str]) -> pd.DataFrame:
    missing = [c for c in key_cols if c not in df.columns]
    if missing:
        return pd.DataFrame([{"dataset": name, "pk": ",".join(key_cols), "pk_dup_rows": np.nan, "note": f"Faltan {missing}"}])
    dups = df.duplicated(subset=key_cols).sum()
    return pd.DataFrame([{"dataset": name, "pk": ",".join(key_cols), "pk_dup_rows": int(dups), "note": ""}])

def fk_orphans_report(child: pd.DataFrame, child_col: str, parent: pd.DataFrame, parent_col: str, name: str) -> pd.DataFrame:
    if child_col not in child.columns or parent_col not in parent.columns:
        return pd.DataFrame([{"relacion": name, "orphans": np.nan, "total": np.nan, "orphans_pct": np.nan, "note": "columna(s) ausente(s)"}])
    child_keys = child[child_col].dropna().astype(str)
    parent_keys = set(parent[parent_col].dropna().astype(str))
    mask_orphan = ~child_keys.isin(parent_keys)
    orphans = int(mask_orphan.sum())
    total = int(len(child_keys))
    return pd.DataFrame([{
        "relacion": name,
        "orphans": orphans,
        "total": total,
        "orphans_pct": round(orphans/total*100, 2) if total else 0.0,
        "note": ""
    }])

def flag_future_dates(dt_series: pd.Series, cutoff: pd.Timestamp = MAX_REASONABLE_DATE) -> pd.Series:
    """Devuelve booleano indicando fechas > cutoff (futuras/sospechosas)."""
    return dt_series.notna() & (dt_series > cutoff)

In [8]:
# === 6) Funciones de limpieza por dataset (encapsulan la lógica) ===

EMAIL_RE = re.compile(r"^[^\s@]+@[^\s@]+\.[^\s@]+$")

def clean_crm(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, int], Dict[str, pd.DataFrame]]:
    """Limpia CRM: IDs, fechas, puntos, tier, emails, duplicados."""
    logs: Dict[str, int] = {}
    extras: Dict[str, pd.DataFrame] = {}

    df = standardize_column_names(df)
    df = strip_strings(df)

    # IDs
    if "customer_id" in df.columns:
        df["customer_id"] = standardize_id(df["customer_id"], upper=True)

    # Normalización de tier
    if "tier_fidelizacion" in df.columns:
        tier = normalize_text(df["tier_fidelizacion"]).str.upper()
        tier = tier.replace({
            "GOLD": "ORO",
            "PLATINUM": "PLATINO",
            "BRONZE": "BRONCE",
            "SILVER": "PLATA",
        })
        df["tier_fidelizacion"] = tier

    # Puntos
    if "puntos_acumulados" in df.columns:
        df["puntos_acumulados_num"] = parse_int(df["puntos_acumulados"])
        # Si puntos negativos, los marcamos como NaN (regla didáctica)
        neg_points = df["puntos_acumulados_num"].notna() & (df["puntos_acumulados_num"] < 0)
        logs["crm_points_negative"] = int(neg_points.sum())
        df.loc[neg_points, "puntos_acumulados_num"] = pd.NA

    # Fechas relevantes
    for col in ["fecha_alta_programa", "fecha_baja", "ultima_actualizacion", "fecha_nacimiento"]:
        if col in df.columns:
            df[col + "_dt"] = parse_datetime_es(df[col])
            logs[f"crm_{col}_future"] = int(flag_future_dates(df[col + "_dt"]).sum())
            # Regla didáctica: fechas futuras -> NaT
            df.loc[flag_future_dates(df[col + "_dt"]), col + "_dt"] = pd.NaT

    # Email: normalizamos a lower y marcamos inválidos
    if "email" in df.columns:
        df["email_norm"] = normalize_text(df["email"]).str.lower()
        invalid = df["email_norm"].notna() & ~df["email_norm"].apply(lambda x: bool(EMAIL_RE.match(str(x))))
        logs["crm_invalid_email"] = int(invalid.sum())
        df.loc[df["email_norm"].isna(), "email_norm"] = np.nan
        # No borramos emails inválidos: los marcamos
        df["email_valido"] = np.where(df["email_norm"].isna(), pd.NA, ~invalid)

    # CP: normalizamos a 5 dígitos cuando se pueda
    if "codigo_postal" in df.columns:
        cp = normalize_text(df["codigo_postal"])
        cp = cp.str.extract(r"(\d{5})", expand=False)
        df["codigo_postal_norm"] = cp

    # Duplicados exactos
    df, removed = drop_exact_duplicates(df)
    logs["crm_exact_dups_removed"] = int(removed)

    # Duplicados por customer_id (conservamos el más reciente si existe ultima_actualizacion_dt)
    if "customer_id" in df.columns:
        sort_col = "ultima_actualizacion_dt" if "ultima_actualizacion_dt" in df.columns else None
        df_dedup, df_dups = dedupe_on_key(df, ["customer_id"], sort_col=sort_col)
        logs["crm_customer_id_dups_removed"] = int(len(df_dups))
        extras["crm_dups_customer_id"] = df_dups
        df = df_dedup

    return df, logs, extras


def clean_productos(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, int], Dict[str, pd.DataFrame]]:
    """Limpia catálogo: IDs, categorías, numéricos, fechas, duplicados."""
    logs: Dict[str, int] = {}
    extras: Dict[str, pd.DataFrame] = {}

    df = standardize_column_names(df)
    df = strip_strings(df)

    if "product_id" in df.columns:
        df["product_id"] = standardize_id(df["product_id"], upper=True)

    # Categóricas (normalización ligera)
    for c in ["categoria", "subcategoria", "marca", "proveedor", "material", "color", "estado_producto"]:
        if c in df.columns:
            df[c] = normalize_text(df[c])

    # Normalizar categoria (ej. 'decoración' -> 'Decoración')
    if "categoria" in df.columns:
        df["categoria"] = df["categoria"].str.strip()
        df["categoria"] = df["categoria"].str.replace(r"\s+", " ", regex=True)
        df["categoria"] = df["categoria"].str.title()

    # Numéricos
    for col in ["precio_venta", "coste_unitario", "peso_kg", "largo_cm", "ancho_cm", "alto_cm", "iva_pct"]:
        if col in df.columns:
            df[col + "_num"] = parse_numeric(df[col])

    # Coste negativo -> NaN (regla didáctica)
    if "coste_unitario_num" in df.columns:
        neg_cost = df["coste_unitario_num"].notna() & (df["coste_unitario_num"] < 0)
        logs["productos_negative_cost"] = int(neg_cost.sum())
        df.loc[neg_cost, "coste_unitario_num"] = np.nan

    # Precio <= 0 -> NaN (regla didáctica)
    if "precio_venta_num" in df.columns:
        bad_price = df["precio_venta_num"].notna() & (df["precio_venta_num"] <= 0)
        logs["productos_price_leq_0"] = int(bad_price.sum())
        df.loc[bad_price, "precio_venta_num"] = np.nan

    # Dims absurdas: alto_cm > 5000 -> NaN (ejemplo)
    if "alto_cm_num" in df.columns:
        absurd = df["alto_cm_num"].notna() & (df["alto_cm_num"] > 5000)
        logs["productos_absurd_height"] = int(absurd.sum())
        df.loc[absurd, "alto_cm_num"] = np.nan

    # Fecha alta catálogo
    if "fecha_alta_catalogo" in df.columns:
        df["fecha_alta_catalogo_dt"] = parse_datetime_es(df["fecha_alta_catalogo"])
        df.loc[flag_future_dates(df["fecha_alta_catalogo_dt"]), "fecha_alta_catalogo_dt"] = pd.NaT

    # Duplicados exactos
    df, removed = drop_exact_duplicates(df)
    logs["productos_exact_dups_removed"] = int(removed)

    # Duplicados por product_id: conservamos el que tenga más info (heurística)
    if "product_id" in df.columns:
        # Heurística: prioriza filas con coste/precio presentes
        df["_score"] = (
            df.get("precio_venta_num", pd.Series([np.nan]*len(df))).notna().astype(int) +
            df.get("coste_unitario_num", pd.Series([np.nan]*len(df))).notna().astype(int)
        )
        df = df.sort_values(["_score"], ascending=False, na_position="last")
        df_dedup, df_dups = dedupe_on_key(df, ["product_id"])
        logs["productos_product_id_dups_removed"] = int(len(df_dups))
        extras["productos_dups_product_id"] = df_dups.drop(columns=["_score"], errors="ignore")
        df = df_dedup.drop(columns=["_score"], errors="ignore")

    return df, logs, extras


def clean_tiendas(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, int], Dict[str, pd.DataFrame]]:
    logs: Dict[str, int] = {}
    extras: Dict[str, pd.DataFrame] = {}

    df = standardize_column_names(df)
    df = strip_strings(df)

    if "store_id" in df.columns:
        df["store_id"] = standardize_id(df["store_id"], upper=True)

    if "fecha_apertura" in df.columns:
        df["fecha_apertura_dt"] = parse_datetime_es(df["fecha_apertura"])
        df.loc[flag_future_dates(df["fecha_apertura_dt"]), "fecha_apertura_dt"] = pd.NaT

    for col in ["metros_cuadrados", "lat", "lon"]:
        if col in df.columns:
            df[col + "_num"] = parse_numeric(df[col])

    # Lat/Lon inválidas
    if "lat_num" in df.columns:
        bad_lat = df["lat_num"].notna() & ((df["lat_num"] < -90) | (df["lat_num"] > 90))
        logs["tiendas_bad_lat"] = int(bad_lat.sum())
        df.loc[bad_lat, "lat_num"] = np.nan

    if "lon_num" in df.columns:
        bad_lon = df["lon_num"].notna() & ((df["lon_num"] < -180) | (df["lon_num"] > 180))
        logs["tiendas_bad_lon"] = int(bad_lon.sum())
        df.loc[bad_lon, "lon_num"] = np.nan

    df, removed = drop_exact_duplicates(df)
    logs["tiendas_exact_dups_removed"] = int(removed)

    if "store_id" in df.columns:
        df_dedup, df_dups = dedupe_on_key(df, ["store_id"])
        logs["tiendas_store_id_dups_removed"] = int(len(df_dups))
        extras["tiendas_dups_store_id"] = df_dups
        df = df_dedup

    return df, logs, extras


def clean_pedidos(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, int], Dict[str, pd.DataFrame]]:
    logs: Dict[str, int] = {}
    extras: Dict[str, pd.DataFrame] = {}

    df = standardize_column_names(df)
    df = strip_strings(df)

    for id_col in ["order_id", "customer_id", "usuario_online_id"]:
        if id_col in df.columns:
            df[id_col] = standardize_id(df[id_col], upper=True)

    if "fecha_pedido" in df.columns:
        df["fecha_pedido_dt"] = parse_datetime_es(df["fecha_pedido"])
        future = flag_future_dates(df["fecha_pedido_dt"])
        logs["pedidos_fecha_future"] = int(future.sum())
        df.loc[future, "fecha_pedido_dt"] = pd.NaT

    for col in ["importe_total", "gastos_envio"]:
        if col in df.columns:
            df[col + "_num"] = parse_numeric(df[col])

    # Duplicados
    df, removed = drop_exact_duplicates(df)
    logs["pedidos_exact_dups_removed"] = int(removed)

    if "order_id" in df.columns:
        sort_col = "ultima_actualizacion" if "ultima_actualizacion" in df.columns else None
        df_dedup, df_dups = dedupe_on_key(df, ["order_id"], sort_col=sort_col)
        logs["pedidos_order_id_dups_removed"] = int(len(df_dups))
        extras["pedidos_dups_order_id"] = df_dups
        df = df_dedup

    return df, logs, extras


def clean_lineas_pedido(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, int], Dict[str, pd.DataFrame]]:
    logs: Dict[str, int] = {}
    extras: Dict[str, pd.DataFrame] = {}

    df = standardize_column_names(df)
    df = strip_strings(df)

    for id_col in ["order_line_id", "order_id", "product_id"]:
        if id_col in df.columns:
            df[id_col] = standardize_id(df[id_col], upper=True)

    if "cantidad" in df.columns:
        df["cantidad_int"] = parse_int(df["cantidad"])
        # Cantidad <=0 se marca como anomalía (no se corrige automáticamente)
        bad_qty = df["cantidad_int"].notna() & (df["cantidad_int"] <= 0)
        logs["lineas_bad_qty_leq0"] = int(bad_qty.sum())

    for col in ["precio_unitario", "descuento_pct", "descuento_importe", "importe_linea", "iva_pct"]:
        if col in df.columns:
            df[col + "_num"] = parse_numeric(df[col])

    df, removed = drop_exact_duplicates(df)
    logs["lineas_exact_dups_removed"] = int(removed)

    if "order_line_id" in df.columns:
        df_dedup, df_dups = dedupe_on_key(df, ["order_line_id"])
        logs["lineas_order_line_id_dups_removed"] = int(len(df_dups))
        extras["lineas_dups_order_line_id"] = df_dups
        df = df_dedup

    return df, logs, extras


def clean_devoluciones_online(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, int], Dict[str, pd.DataFrame]]:
    logs: Dict[str, int] = {}
    extras: Dict[str, pd.DataFrame] = {}

    df = standardize_column_names(df)
    df = strip_strings(df)

    for id_col in ["return_id", "order_id", "order_line_id", "product_id"]:
        if id_col in df.columns:
            df[id_col] = standardize_id(df[id_col], upper=True)

    if "fecha_devolucion" in df.columns:
        df["fecha_devolucion_dt"] = parse_datetime_es(df["fecha_devolucion"])
        future = flag_future_dates(df["fecha_devolucion_dt"])
        logs["devol_online_fecha_future"] = int(future.sum())
        df.loc[future, "fecha_devolucion_dt"] = pd.NaT

    if "cantidad_devuelta" in df.columns:
        df["cantidad_devuelta_int"] = parse_int(df["cantidad_devuelta"])
        bad_qty = df["cantidad_devuelta_int"].notna() & (df["cantidad_devuelta_int"] <= 0)
        logs["devol_online_bad_qty_leq0"] = int(bad_qty.sum())

    if "importe_reembolsado" in df.columns:
        df["importe_reembolsado_num"] = parse_numeric(df["importe_reembolsado"])
        neg = df["importe_reembolsado_num"].notna() & (df["importe_reembolsado_num"] < 0)
        logs["devol_online_negative_refund"] = int(neg.sum())

    df, removed = drop_exact_duplicates(df)
    logs["devol_online_exact_dups_removed"] = int(removed)

    # Dedup por return_id+product_id (si aplica)
    key = [c for c in ["return_id", "product_id"] if c in df.columns]
    if len(key) == 2:
        df_dedup, df_dups = dedupe_on_key(df, key)
        logs["devol_online_dups_removed"] = int(len(df_dups))
        extras["devol_online_dups"] = df_dups
        df = df_dedup

    return df, logs, extras


def clean_ventas_pos(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, int], Dict[str, pd.DataFrame]]:
    logs: Dict[str, int] = {}
    extras: Dict[str, pd.DataFrame] = {}

    df = standardize_column_names(df)
    df = strip_strings(df)

    for id_col in ["ticket_line_id", "ticket_id", "store_id", "caja_id", "cajero_id", "customer_id", "product_id"]:
        if id_col in df.columns:
            upper = id_col in {"ticket_line_id","ticket_id","store_id","caja_id","cajero_id","customer_id","product_id"}
            df[id_col] = standardize_id(df[id_col], upper=upper)

    if "fecha_hora" in df.columns:
        df["fecha_hora_dt"] = parse_datetime_es(df["fecha_hora"])
        future = flag_future_dates(df["fecha_hora_dt"])
        logs["ventas_pos_fecha_future"] = int(future.sum())
        df.loc[future, "fecha_hora_dt"] = pd.NaT

    if "cantidad" in df.columns:
        df["cantidad_int"] = parse_int(df["cantidad"])
        bad_qty = df["cantidad_int"].notna() & (df["cantidad_int"] <= 0)
        logs["ventas_pos_bad_qty_leq0"] = int(bad_qty.sum())

    for col in ["precio_unitario", "descuento_pct", "descuento_importe", "importe_linea"]:
        if col in df.columns:
            df[col + "_num"] = parse_numeric(df[col])

    df, removed = drop_exact_duplicates(df)
    logs["ventas_pos_exact_dups_removed"] = int(removed)

    if "ticket_line_id" in df.columns:
        df_dedup, df_dups = dedupe_on_key(df, ["ticket_line_id"])
        logs["ventas_pos_ticket_line_id_dups_removed"] = int(len(df_dups))
        extras["ventas_pos_dups_ticket_line_id"] = df_dups
        df = df_dedup

    return df, logs, extras


def clean_devoluciones_tienda(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, int], Dict[str, pd.DataFrame]]:
    logs: Dict[str, int] = {}
    extras: Dict[str, pd.DataFrame] = {}

    df = standardize_column_names(df)
    df = strip_strings(df)

    for id_col in ["return_id", "store_id", "ticket_id_original", "ticket_line_id_original", "order_id_original", "customer_id", "product_id"]:
        if id_col in df.columns:
            df[id_col] = standardize_id(df[id_col], upper=True)

    if "fecha_devolucion" in df.columns:
        df["fecha_devolucion_dt"] = parse_datetime_es(df["fecha_devolucion"])
        future = flag_future_dates(df["fecha_devolucion_dt"])
        logs["devol_tienda_fecha_future"] = int(future.sum())
        df.loc[future, "fecha_devolucion_dt"] = pd.NaT

    if "cantidad_devuelta" in df.columns:
        df["cantidad_devuelta_int"] = parse_int(df["cantidad_devuelta"])
        bad_qty = df["cantidad_devuelta_int"].notna() & (df["cantidad_devuelta_int"] <= 0)
        logs["devol_tienda_bad_qty_leq0"] = int(bad_qty.sum())

    if "importe_reembolsado" in df.columns:
        df["importe_reembolsado_num"] = parse_numeric(df["importe_reembolsado"])
        neg = df["importe_reembolsado_num"].notna() & (df["importe_reembolsado_num"] < 0)
        logs["devol_tienda_negative_refund"] = int(neg.sum())

    df, removed = drop_exact_duplicates(df)
    logs["devol_tienda_exact_dups_removed"] = int(removed)

    key = [c for c in ["return_id", "product_id"] if c in df.columns]
    if len(key) == 2:
        df_dedup, df_dups = dedupe_on_key(df, key)
        logs["devol_tienda_dups_removed"] = int(len(df_dups))
        extras["devol_tienda_dups"] = df_dups
        df = df_dedup

    return df, logs, extras


def clean_pagos_tienda(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, int], Dict[str, pd.DataFrame]]:
    logs: Dict[str, int] = {}
    extras: Dict[str, pd.DataFrame] = {}

    df = standardize_column_names(df)
    df = strip_strings(df)

    for id_col in ["payment_id", "ticket_id", "store_id", "caja_id", "cajero_id"]:
        if id_col in df.columns:
            df[id_col] = standardize_id(df[id_col], upper=True)

    if "fecha_pago" in df.columns:
        df["fecha_pago_dt"] = parse_datetime_es(df["fecha_pago"])
        future = flag_future_dates(df["fecha_pago_dt"])
        logs["pagos_fecha_future"] = int(future.sum())
        df.loc[future, "fecha_pago_dt"] = pd.NaT

    if "importe_pagado" in df.columns:
        df["importe_pagado_num"] = parse_numeric(df["importe_pagado"])
        neg = df["importe_pagado_num"].notna() & (df["importe_pagado_num"] < 0)
        logs["pagos_negative_amount"] = int(neg.sum())

    # Normalizar método de pago
    if "metodo_pago" in df.columns:
        mp = normalize_text(df["metodo_pago"]).str.upper()
        mp = mp.replace({
            "TARJETA CREDITO": "TARJETA",
            "TARJETA CRÉDITO": "TARJETA",
            "CREDIT CARD": "TARJETA",
            "CARD": "TARJETA",
            "EFECTIVO ": "EFECTIVO",
        })
        df["metodo_pago_norm"] = mp

    df, removed = drop_exact_duplicates(df)
    logs["pagos_exact_dups_removed"] = int(removed)

    if "payment_id" in df.columns:
        df_dedup, df_dups = dedupe_on_key(df, ["payment_id"])
        logs["pagos_payment_id_dups_removed"] = int(len(df_dups))
        extras["pagos_dups_payment_id"] = df_dups
        df = df_dedup

    return df, logs, extras


def clean_stock_diario(df: pd.DataFrame) -> Tuple[pd.DataFrame, Dict[str, int], Dict[str, pd.DataFrame]]:
    logs: Dict[str, int] = {}
    extras: Dict[str, pd.DataFrame] = {}

    df = standardize_column_names(df)
    df = strip_strings(df)

    for id_col in ["store_id", "product_id"]:
        if id_col in df.columns:
            df[id_col] = standardize_id(df[id_col], upper=True)

    if "fecha" in df.columns:
        df["fecha_dt"] = parse_datetime_es(df["fecha"])
        future = flag_future_dates(df["fecha_dt"])
        logs["stock_fecha_future"] = int(future.sum())
        df.loc[future, "fecha_dt"] = pd.NaT

    for col in ["stock_cierre", "stock_reservado", "stock_en_transito", "stock_minimo"]:
        if col in df.columns:
            df[col + "_int"] = parse_int(df[col])

    # Stock negativo -> 0 (regla didáctica) pero guardamos flag
    if "stock_cierre_int" in df.columns:
        neg = df["stock_cierre_int"].notna() & (df["stock_cierre_int"] < 0)
        logs["stock_negative_stock_cierre"] = int(neg.sum())
        df["stock_cierre_corrected"] = False
        df.loc[neg, "stock_cierre_int"] = 0
        df.loc[neg, "stock_cierre_corrected"] = True

    df, removed = drop_exact_duplicates(df)
    logs["stock_exact_dups_removed"] = int(removed)

    # Dedup por (fecha_dt, store_id, product_id) si existe
    key = [c for c in ["fecha_dt", "store_id", "product_id"] if c in df.columns]
    if len(key) == 3:
        df_dedup, df_dups = dedupe_on_key(df, key)
        logs["stock_key_dups_removed"] = int(len(df_dups))
        extras["stock_dups_key"] = df_dups
        df = df_dedup

    return df, logs, extras

In [17]:
# === 7) Carga de datasets (raw) ===

dfs_raw: Dict[str, pd.DataFrame] = {}
for name, fname in FILES.items():
    path = DATA_DIR / fname
    if not path.exists():
        raise FileNotFoundError(f"No encuentro {path}. Coloca los CSV en {DATA_DIR.resolve()} o ajusta DATA_DIR.")
    dfs_raw[name] = load_csv_raw(path)

# Resumen inicial
reports_before = pd.concat([basic_shape_report(df, name) for name, df in dfs_raw.items()], ignore_index=True)
reports_before.sort_values("dataset")

,dataset,rows,cols,null_cells,null_cells_pct,exact_dups
0,crm,103,20,214,10.39,1
5,devoluciones_online,224,12,243,9.04,0
7,devoluciones_tienda,211,16,516,15.28,0
4,lineas_pedido,1949,12,1875,8.02,8
8,pagos_tienda,1646,12,2010,10.18,10
3,pedidos,656,16,831,7.92,2
1,productos,101,18,18,0.99,0
9,stock_diario,1940,9,2015,11.54,14
2,tiendas,8,19,6,3.95,0
6,ventas_pos,2521,19,4534,9.47,30


In [18]:
reports_before

,dataset,rows,cols,null_cells,null_cells_pct,exact_dups
0,crm,103,20,214,10.39,1
1,productos,101,18,18,0.99,0
2,tiendas,8,19,6,3.95,0
3,pedidos,656,16,831,7.92,2
4,lineas_pedido,1949,12,1875,8.02,8
5,devoluciones_online,224,12,243,9.04,0
6,ventas_pos,2521,19,4534,9.47,30
7,devoluciones_tienda,211,16,516,15.28,0
8,pagos_tienda,1646,12,2010,10.18,10
9,stock_diario,1940,9,2015,11.54,14


In [19]:
# === 8) Aplicar limpieza + recopilar logs y datasets extra (duplicados / rechazados) ===

cleaned: Dict[str, pd.DataFrame] = {}
extras_all: Dict[str, pd.DataFrame] = {}
logs_all: Dict[str, Dict[str, int]] = {}

cleaned["crm"], logs_all["crm"], extras = clean_crm(dfs_raw["crm"])
extras_all.update({f"crm__{k}": v for k, v in extras.items()})

cleaned["productos"], logs_all["productos"], extras = clean_productos(dfs_raw["productos"])
extras_all.update({f"productos__{k}": v for k, v in extras.items()})

cleaned["tiendas"], logs_all["tiendas"], extras = clean_tiendas(dfs_raw["tiendas"])
extras_all.update({f"tiendas__{k}": v for k, v in extras.items()})

cleaned["pedidos"], logs_all["pedidos"], extras = clean_pedidos(dfs_raw["pedidos"])
extras_all.update({f"pedidos__{k}": v for k, v in extras.items()})

cleaned["lineas_pedido"], logs_all["lineas_pedido"], extras = clean_lineas_pedido(dfs_raw["lineas_pedido"])
extras_all.update({f"lineas_pedido__{k}": v for k, v in extras.items()})

cleaned["devoluciones_online"], logs_all["devoluciones_online"], extras = clean_devoluciones_online(dfs_raw["devoluciones_online"])
extras_all.update({f"devoluciones_online__{k}": v for k, v in extras.items()})

cleaned["ventas_pos"], logs_all["ventas_pos"], extras = clean_ventas_pos(dfs_raw["ventas_pos"])
extras_all.update({f"ventas_pos__{k}": v for k, v in extras.items()})

cleaned["devoluciones_tienda"], logs_all["devoluciones_tienda"], extras = clean_devoluciones_tienda(dfs_raw["devoluciones_tienda"])
extras_all.update({f"devoluciones_tienda__{k}": v for k, v in extras.items()})

cleaned["pagos_tienda"], logs_all["pagos_tienda"], extras = clean_pagos_tienda(dfs_raw["pagos_tienda"])
extras_all.update({f"pagos_tienda__{k}": v for k, v in extras.items()})

cleaned["stock_diario"], logs_all["stock_diario"], extras = clean_stock_diario(dfs_raw["stock_diario"])
extras_all.update({f"stock_diario__{k}": v for k, v in extras.items()})

# Resumen de logs
logs_flat = []
for ds, d in logs_all.items():
    for k, v in d.items():
        logs_flat.append({"dataset": ds, "metric": k, "value": v})
logs_df = pd.DataFrame(logs_flat).sort_values(["dataset", "metric"])
logs_df.head(30)

C:\Users\anton\AppData\Local\Temp\ipykernel_25280\849750928.py:45: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dt1 = pd.to_datetime(s_norm, errors="coerce", dayfirst=dayfirst, utc=False)
C:\Users\anton\AppData\Local\Temp\ipykernel_25280\849750928.py:45: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt1 = pd.to_datetime(s_norm, errors="coerce", dayfirst=dayfirst, utc=False)
C:\Users\anton\AppData\Local\Temp\ipykernel_25280\849750928.py:45: UserWarning: Parsing dates in %Y-%m-%d format when dayfirst=True was specified. Pass `dayfirst=False` or specify a format to silence this warning.
  dt1 = pd.to_datetime(s_norm, errors="coerce", dayfirst=dayfirst, utc=False)
C:\Users\anton\AppData\Local\Temp\ipykernel_25280\849750928.py:45: UserWarning: Parsing

,dataset,metric,value
7,crm,crm_customer_id_dups_removed,2
6,crm,crm_exact_dups_removed,1
1,crm,crm_fecha_alta_programa_future,0
2,crm,crm_fecha_baja_future,0
4,crm,crm_fecha_nacimiento_future,0
5,crm,crm_invalid_email,4
0,crm,crm_points_negative,3
3,crm,crm_ultima_actualizacion_future,0
24,devoluciones_online,devol_online_bad_qty_leq0,2
27,devoluciones_online,devol_online_dups_removed,0


In [20]:
# Resumen de tamaños después de limpieza
reports_after = pd.concat([basic_shape_report(df, name) for name, df in cleaned.items()], ignore_index=True)
reports_after.sort_values("dataset")

,dataset,rows,cols,null_cells,null_cells_pct,exact_dups
0,crm,100,28,447,15.96,0
5,devoluciones_online,224,15,426,12.68,0
7,devoluciones_tienda,205,19,692,17.77,0
4,lineas_pedido,1929,18,3247,9.35,0
8,pagos_tienda,1631,15,3119,12.75,0
3,pedidos,650,19,1303,10.55,0
1,productos,100,26,26,1.00,0
9,stock_diario,626,15,1304,13.89,0
2,tiendas,7,23,11,6.83,0
6,ventas_pos,2481,25,4905,7.91,0


In [21]:
# Comparativa before vs after
cmp = reports_before.merge(reports_after, on="dataset", suffixes=("_before", "_after"))
cmp[["dataset","rows_before","rows_after","exact_dups_before","exact_dups_after","null_cells_pct_before","null_cells_pct_after"]].sort_values("dataset")

,dataset,rows_before,rows_after,exact_dups_before,exact_dups_after,null_cells_pct_before,null_cells_pct_after
0,crm,103,100,1,0,10.39,15.96
5,devoluciones_online,224,224,0,0,9.04,12.68
7,devoluciones_tienda,211,205,0,0,15.28,17.77
4,lineas_pedido,1949,1929,8,0,8.02,9.35
8,pagos_tienda,1646,1631,10,0,10.18,12.75
3,pedidos,656,650,2,0,7.92,10.55
1,productos,101,100,0,0,0.99,1.00
9,stock_diario,1940,626,14,0,11.54,13.89
2,tiendas,8,7,0,0,3.95,6.83
6,ventas_pos,2521,2481,30,0,9.47,7.91


In [22]:
# === 9) Consistencia básica después de limpiar (PK/FK + conciliaciones simples) ===

pk_checks = pd.concat([
    pk_duplicate_report(cleaned["crm"], "crm", ["customer_id"]),
    pk_duplicate_report(cleaned["productos"], "productos", ["product_id"]),
    pk_duplicate_report(cleaned["tiendas"], "tiendas", ["store_id"]),
    pk_duplicate_report(cleaned["pedidos"], "pedidos", ["order_id"]),
    pk_duplicate_report(cleaned["lineas_pedido"], "lineas_pedido", ["order_line_id"]),
    pk_duplicate_report(cleaned["ventas_pos"], "ventas_pos", ["ticket_line_id"]),
    pk_duplicate_report(cleaned["pagos_tienda"], "pagos_tienda", ["payment_id"]),
], ignore_index=True)

pk_checks

,dataset,pk,pk_dup_rows,note
0,crm,customer_id,0,
1,productos,product_id,0,
2,tiendas,store_id,0,
3,pedidos,order_id,0,
4,lineas_pedido,order_line_id,0,
5,ventas_pos,ticket_line_id,0,
6,pagos_tienda,payment_id,0,


In [23]:
# FK checks (huérfanos)
fk_checks = pd.concat([
    fk_orphans_report(cleaned["pedidos"], "customer_id", cleaned["crm"], "customer_id", "pedidos.customer_id -> crm.customer_id"),
    fk_orphans_report(cleaned["lineas_pedido"], "order_id", cleaned["pedidos"], "order_id", "lineas.order_id -> pedidos.order_id"),
    fk_orphans_report(cleaned["lineas_pedido"], "product_id", cleaned["productos"], "product_id", "lineas.product_id -> productos.product_id"),
    fk_orphans_report(cleaned["ventas_pos"], "product_id", cleaned["productos"], "product_id", "ventas_pos.product_id -> productos.product_id"),
    fk_orphans_report(cleaned["ventas_pos"], "store_id", cleaned["tiendas"], "store_id", "ventas_pos.store_id -> tiendas.store_id"),
    fk_orphans_report(cleaned["pagos_tienda"], "ticket_id", cleaned["ventas_pos"], "ticket_id", "pagos.ticket_id -> ventas_pos.ticket_id (ojo: ventas_pos está a nivel línea)"),
    fk_orphans_report(cleaned["stock_diario"], "product_id", cleaned["productos"], "product_id", "stock.product_id -> productos.product_id"),
    fk_orphans_report(cleaned["stock_diario"], "store_id", cleaned["tiendas"], "store_id", "stock.store_id -> tiendas.store_id"),
], ignore_index=True)

fk_checks

,relacion,orphans,total,orphans_pct,note
0,pedidos.customer_id -> crm.customer_id,48,490,9.80,
1,lineas.order_id -> pedidos.order_id,0,1929,0.00,
2,lineas.product_id -> productos.product_id,62,1899,3.26,
3,ventas_pos.product_id -> productos.product_id,74,2481,2.98,
4,ventas_pos.store_id -> tiendas.store_id,31,2449,1.27,
5,pagos.ticket_id -> ventas_pos.ticket_id (ojo: ...,10,1626,0.62,
6,stock.product_id -> productos.product_id,5,624,0.80,
7,stock.store_id -> tiendas.store_id,35,624,5.61,


In [24]:
# Conciliación didáctica: pedidos (importe_total_num) vs suma de lineas (importe_linea_num)
# Nota: Con errores intencionales, no esperamos 100% matching.
if "importe_total_num" in cleaned["pedidos"].columns and "importe_linea_num" in cleaned["lineas_pedido"].columns:
    lines_sum = (
        cleaned["lineas_pedido"]
        .groupby("order_id", dropna=False)["importe_linea_num"]
        .sum(min_count=1)
        .rename("importe_lineas_sum")
        .reset_index()
    )
    ped = cleaned["pedidos"][["order_id", "importe_total_num"]].merge(lines_sum, on="order_id", how="left")
    ped["diff"] = ped["importe_total_num"] - ped["importe_lineas_sum"]
    ped["abs_diff"] = ped["diff"].abs()
    conc = ped["abs_diff"].dropna()
    print("Pedidos con diff no nulo:", int(conc.notna().sum()))
    print("Pct con diff > 1 EUR:", round((conc > 1).mean()*100, 2))
    ped.sort_values("abs_diff", ascending=False).head(10)
else:
    print("No encuentro columnas numéricas necesarias para conciliación.")

Pedidos con diff no nulo: 631
Pct con diff > 1 EUR: 80.19


In [25]:
# Conciliación didáctica: suma importe_linea en POS vs suma importe_pagado en pagos_tienda por ticket_id
if "importe_linea_num" in cleaned["ventas_pos"].columns and "importe_pagado_num" in cleaned["pagos_tienda"].columns:
    pos_tot = cleaned["ventas_pos"].groupby("ticket_id")["importe_linea_num"].sum(min_count=1).rename("pos_total").reset_index()
    pay_tot = cleaned["pagos_tienda"].groupby("ticket_id")["importe_pagado_num"].sum(min_count=1).rename("pagos_total").reset_index()
    recon = pos_tot.merge(pay_tot, on="ticket_id", how="outer")
    recon["diff"] = recon["pos_total"] - recon["pagos_total"]
    recon["abs_diff"] = recon["diff"].abs()
    recon.sort_values("abs_diff", ascending=False).head(10)
else:
    print("No encuentro columnas numéricas necesarias para conciliación POS vs Pagos.")

In [26]:
# === 10) Exportación (datasets limpios + extras + logs) ===

# 10.1. Exportar datasets limpios
for name, df in cleaned.items():
    out_path = OUT_DIR / f"{name}_clean.csv"
    df.to_csv(out_path, index=False, encoding="utf-8")
    print("OK ->", out_path)

# 10.2. Exportar datasets extra (duplicados / descartados) para prácticas
EXTRAS_DIR = OUT_DIR / "extras"
EXTRAS_DIR.mkdir(parents=True, exist_ok=True)

for name, df in extras_all.items():
    out_path = EXTRAS_DIR / f"{name}.csv"
    df.to_csv(out_path, index=False, encoding="utf-8")
    print("EXTRA ->", out_path)

# 10.3. Exportar logs
logs_df.to_csv(OUT_DIR / "cleaning_metrics.csv", index=False, encoding="utf-8")
with open(OUT_DIR / "cleaning_metrics.json", "w", encoding="utf-8") as f:
    json.dump(logs_all, f, ensure_ascii=False, indent=2)

# 10.4. Exportar comparativa before/after
cmp.to_csv(OUT_DIR / "before_after_summary.csv", index=False, encoding="utf-8")

print("Export finalizado en:", OUT_DIR.resolve())

OK -> outputs\crm_clean.csv
OK -> outputs\productos_clean.csv
OK -> outputs\tiendas_clean.csv
OK -> outputs\pedidos_clean.csv
OK -> outputs\lineas_pedido_clean.csv
OK -> outputs\devoluciones_online_clean.csv
OK -> outputs\ventas_pos_clean.csv
OK -> outputs\devoluciones_tienda_clean.csv
OK -> outputs\pagos_tienda_clean.csv
OK -> outputs\stock_diario_clean.csv
EXTRA -> outputs\extras\crm__crm_dups_customer_id.csv
EXTRA -> outputs\extras\productos__productos_dups_product_id.csv
EXTRA -> outputs\extras\tiendas__tiendas_dups_store_id.csv
EXTRA -> outputs\extras\pedidos__pedidos_dups_order_id.csv
EXTRA -> outputs\extras\lineas_pedido__lineas_dups_order_line_id.csv
EXTRA -> outputs\extras\devoluciones_online__devol_online_dups.csv
EXTRA -> outputs\extras\ventas_pos__ventas_pos_dups_ticket_line_id.csv
EXTRA -> outputs\extras\devoluciones_tienda__devol_tienda_dups.csv
EXTRA -> outputs\extras\pagos_tienda__pagos_dups_payment_id.csv
EXTRA -> outputs\extras\stock_diario__stock_dups_key.csv
Export 

## Ejercicios / propuestas para el aula

1) **Comparar antes vs después**  
   - ¿Cuántos duplicados exactos se eliminaron por dataset?  
   - ¿Qué campos siguen teniendo altos porcentajes de nulos y por qué?

2) **Decisiones de negocio**  
   - ¿Debemos imputar `email` faltante? ¿O es mejor mantener nulo?  
   - ¿Un `precio_venta` = 0 se corrige, se descarta o se marca?

3) **Excepciones y trazabilidad**  
   - Revisa los CSV dentro de `output_clean/extras/`: ¿qué tipo de errores aparecen?  
   - Define una estrategia: **descartar** vs **corregir** vs **mantener con flag**.

4) **Preparación para Notebook 4 (Integración)**  
   - Con los datasets `*_clean.csv`, intenta mejorar los checks de FK (huérfanos).  
   - Decide cómo tratar transacciones con `customer_id` no encontrado (GUEST vs error).

> Siguiente notebook recomendado: **Notebook 4 – Integración y fact table**.